In [2]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd

sys.path.append(os.path.abspath(os.getcwd() + '/database'))
from db_api import *

db = db_api()

In [3]:
stats = db.get_items('stats')
for stat in stats:
    print stat.name

QUEUE_LEN
ITEMS_PROCESSED
EXEC_TIME
IDLE_TIME
MEMORY_ALLOCATED
NUM_STATES


In [4]:
msus = db.get_items('msus')
for msu in msus:
    print 'ID: ' + str(msu.msu)# + ', TYPE: ' + msu.msu_type.name

ID: 10
ID: 11
ID: 12
ID: 13
ID: 14
ID: 15
ID: 16
ID: 17
ID: 18


In [5]:
ts = db.get_msu_timeseries(msus[0])
for t in ts:
    print t.statistic.name

QUEUE_LEN
ITEMS_PROCESSED
MEMORY_ALLOCATED
NUM_STATES
EXEC_TIME
IDLE_TIME


In [6]:
df = db.get_msu_full_df(msus[0])
print df

     EXEC_TIME IDLE_TIME ITEMS_PROCESSED MEMORY_ALLOCATED NUM_STATES  \
0            0         0               0                0          0   
1            1         0               1                0          0   
2            1         0               1                0          0   
3            1         0               2                0          0   
4            1         0               3                0          0   
5            1         0               4                0          0   
6            1         0               5                0          0   
7            1         0               6                0          0   
8            1         0               7                0          0   
9            1         0               7                0          0   
10           1         0               8                0          0   
11           1         0               9                0          0   
12           1         0              10                0       